In [1]:
import pandas as pd

In [2]:
songs = pd.read_csv('hipHopLyrics.csv')

songs analisis:

In [3]:
songs.head()

,ALink,SName,SLink,Lyric,Idiom
0,/beastie-boys/,Sabotage,/beastie-boys/sabotage.html,"I can't stand it, I know you planned it. I'm a...",ENGLISH
1,/beastie-boys/,Intergalactic,/beastie-boys/intergalactic.html,Intergalactic Planetary Planetary Intergalacti...,ENGLISH
2,/beastie-boys/,(You Gotta) Fight For Your Right (To Party!),/beastie-boys/you-gotta-fight-for-your-right-t...,Kick it!. [Verse 1]. You wake up late for scho...,ENGLISH
3,/beastie-boys/,Sure Shot,/beastie-boys/sure-shot.html,"You can't, you won't and you don't stop. Mike ...",ENGLISH
4,/beastie-boys/,No Sleep 'Till Brooklyn,/beastie-boys/no-sleep-till-brooklyn.html,(chorus) No sleep 'til - Brooklyn. . Foot on t...,ENGLISH


In [4]:
len(songs)

23108

lyric example

In [5]:
songs["Lyric"].iloc[3]

"You can't, you won't and you don't stop. Mike d come on and rock the sure shot. I've got the brand new doo-doo guaranteed. Like yoo hoo. I'm on like dr john, yea mr zu zu. I'm a newlywed, not a divorcee. And everything I do is funky like lee dorsey. Well, it's the taking fo pelham, one, two, three. If you want a doodoo rhyme then come see me. I've got the savoir faire with the unique rhyme. And. I keep it on and on, it's never quitting time and. Strictly hand held is the style I go. Never rock the mic with the panty hose. I strap on my ear goggles and I'm ready to go. 'couse at the boards is the man they call the. Mario. Pull up at the function and you know I kojak. To all the party people that are on my bozak. I've got more action than my man john woo. And I've got mad hits like I was rod crew. You can't, you won't and you don't stop. Ad rock come and rock the sure shot. Hurricane will cross fade on your ass and. Bust your ear drums. Listen everybody 'couse I'm shifting gears I'm. Fr

### now we define the rhyme scheme 
(we will use the optimized version from https://github.com/daviddelapuente/rhymeScheme)

In [6]:
import nltk
nltk.download('cmudict')

[nltk_data] Downloading package cmudict to /home/dania/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

In [7]:
#this function takes 2 strings, and return True if one of them is sufix of the other
def isSufix(s1,s2):
    if(len(s1)>=len(s2)):
        for i in range(len(s2)):
            if (s2[len(s2)-1-i]!=s1[len(s1)-1-i]):
                return False
    else:
        for i in range(len(s1)):
            if(s1[len(s1)-1-i]!=s2[len(s2)-1-i]):
                return False
    return True



phone_dictionaryOPT = nltk.corpus.cmudict.dict()
def possible_phones_OPT(word):
    if word not in phone_dictionaryOPT:
        return []
    return phone_dictionaryOPT[word]





def first_score(phones1,phones2):
    if(len(phones1)==0 or len(phones2)==0):
        return 0
    phones1=phones1[::-1]
    phones2=phones2[::-1]
    if len(phones1)>len(phones2):
        phones1,phones2=phones2,phones1
    hit=0
    total=0
    for i in range(len(phones1)):
        if phones1[i]==phones2[i]:
            if hasDigit(phones1[i]):
                hit+=2
                total+=2
            else:
                hit+=1
                total+=1
        else:
            total+=1
    return hit/total




def hasDigit(s):
    for char in s:
        if char.isdigit():
            return True
    return False



def rhyme_scoreOPT(first_word, second_word):
    return getScore(first_word,second_word,possible_phones_OPT)



#this function was inspired by word_similarity from https://github.com/kevin-brown/rhyme-detect
def getScore(first_word, second_word,dicrtionaryFunction):
    #first we filter some border cases like empty strings
    if(first_word==None or second_word==None):
        return 0
    if(len(first_word)==0 or len(second_word)==0):
        return 0
    if (isSufix(first_word,second_word)):
        return 0
    

    #this build the phonemas
    first_phones = dicrtionaryFunction(first_word)
    second_phones = dicrtionaryFunction(second_word)
    if not first_phones or not second_phones:
        return 0

    #we invert the array
    first_range = first_phones[0][::-1]
    second_range = second_phones[0][::-1]

    #we put the smaller array in first_range
    if len(first_range) > len(second_range):
        first_range, second_range = second_range, first_range

    hits = 0
    total = len(first_range)

    c1=[]
    b=False
    for i in range(len(first_range)):
        phone=first_range[i]
        if(hasDigit(phone)):
            b=True
        c1.append(phone)
        if(not phone[-1].isdigit() and b):
            break

    c2=[]
    b=False
    for i in range(len(second_range)):
        phone=second_range[i]
        if(hasDigit(phone)):
            b=True
        c2.append(phone)
        if(not phone[-1].isdigit() and b):
            break
    
    score1=first_score(c1,c2)
    return score1


examples of the scheme

In [11]:
print(rhyme_scoreOPT("commit","submit"))
print(rhyme_scoreOPT("suffix","prefix"))
print(rhyme_scoreOPT("suffix","phoenix"))
print(rhyme_scoreOPT("suffix","helix"))
print(rhyme_scoreOPT("octopus","bus"))
print(rhyme_scoreOPT("car","palm"))
print(rhyme_scoreOPT("car","bit"))

1.0
1.0
0.8
0.8
0.3333333333333333
0.5
0.0


now we define the functions that will create the challenge-response

In [33]:
#this function take a song, and a treshold (an integer)
#and what is does is to split the song (that actually is a string)
#and create a list where each positiono of the list contains a line of the song.
#the reason why it doesent use the .split(".") function. is because there are some lines that has
#points in between. and that the reason why it use a treshold (is like if the substring between two "."
# is less than the treashold it cant be consider a line)
def makeLines(song,treshold):
    lines=[]
    i=0
    for j in range(len(song)):
        if song[j]==".":
            if j-i<treshold:
                if len(lines)>0:
                    lines[len(lines)-1]+=song[i:j]
                    i=j+1
            else:
                lines.append(song[i:j])
                i=j+1
    return lines

def takeLastWord(line):
    words=line.split(" ")
    return words[len(words)-1]

def challengeResponseGeneratorFromSong(song,treshold=10):
    L=[]
    lines=makeLines(song,treshold)
    for i in range(len(lines)-1):
        word1=takeLastWord(lines[i])
        word2=takeLastWord(lines[i+1])
        score = rhyme_scoreOPT(word1,word2)
        if score>0.34:
            v=[]
            v.append(lines[i])
            v.append(lines[i+1])
            L.append(v)
    return L

def challengeResponseGeneratorFromAllSongs(songs,treshold=10):
    L=[]
    for s in songs:
        L+=challengeResponseGeneratorFromSong(s,treshold)
    return L

In [34]:
crList=challengeResponseGeneratorFromAllSongs(songs["Lyric"],10)

In [35]:
len(crList)

184317

In [36]:
crdataframe = pd.DataFrame(crList,columns=['challenge','response'])

In [37]:
crdataframe.head()

,challenge,response
0,So while you sit back and wonder why,I got this fucking thorn in my side
1,'Cause what you see you might not get,And we can bet so don't you get souped yet
2,I can't stand rockin' when I'm in this place,Because I feel disgraced because you're all i...
3,But make no mistakes and switch up my channel,I'm buddy rich when I fly off the handle
4,Don't you tell me to smile,You stick around I'll make it worth your while


In [38]:
crdataframe.to_csv(r'challengeResponse', index = False, header=True)